### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers.recurrent import GRU
from keras.layers import Dense, LSTM

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.ascii_uppercase
encrypt = vigenere_cipher
samples_count = 1000000
text_length = 5

set_characters(letters)
model_path = f'{encrypt.__name__}_{len(letters)}x{text_length}_best_model.h5'

In [13]:
model_path

'vigenere_cipher_26x5_best_model.h5'

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)

train_cipher = to_vec(map(encrypt, text))
train_text = to_vec(text)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(GRU(128, activation='relu', input_shape=train_cipher.shape[1:], return_sequences=True))
model.add(GRU(128, activation='relu', return_sequences=True))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [5]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='auto'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]

In [6]:
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=5, callbacks=callbacks)

Train on 900000 samples, validate on 100000 samples
Epoch 1/5
900000/900000 [==============================] - 776s 862us/step - loss: 6.4972 - acc: 0.5658 - val_loss: 0.3696 - val_acc: 0.9081

Epoch 00001: val_acc improved from -inf to 0.90814, saving model to vigenere_cipher_26x5_best_model.h5
Epoch 2/5
900000/900000 [==============================] - 773s 859us/step - loss: 4.1492 - acc: 0.6967 - val_loss: 12.7690 - val_acc: 0.4949

Epoch 00002: val_acc did not improve from 0.90814
Epoch 3/5
900000/900000 [==============================] - 751s 834us/step - loss: 3.4624 - acc: 0.7915 - val_loss: 0.0119 - val_acc: 0.9998

Epoch 00003: val_acc improved from 0.90814 to 0.99981, saving model to vigenere_cipher_26x5_best_model.h5
Epoch 4/5
900000/900000 [==============================] - 733s 815us/step - loss: 2.1978 - acc: 0.8679 - val_loss: 0.2687 - val_acc: 0.9050

Epoch 00004: val_acc did not improve from 0.99981
Epoch 5/5
900000/900000 [==============================] - 776s 862us/

### Predicting and Evaluating the Model

In [7]:
model = load_model(model_path)

In [8]:
test_cyhper = 'TGFYW'
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: SECUR
Test cipher vs encrypted prediction: TGFYW vs TGFYW
